In [4]:
import sys
import os

# Agrega la carpeta raíz del proyecto al path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


### Exploración de datos con SQL Alchemy

In [18]:
import pandas as pd
from core.connection import get_engine
from core.queries import QUERIES
from core.connection_notebook import get_engine_notebook

%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
engine = get_engine_notebook()

### Consultas

In [14]:
query = """
WITH TotalCounts AS (
    SELECT 
        Year_Month, 
        COUNT(*) AS TotalScrobblings
    FROM Clean_LastfmData 
    GROUP BY Year_Month
),
RankedArtists AS (
    SELECT 
        Year_Month,
        Artist, 
        COUNT(*) AS Scrobblings,
        ROW_NUMBER() OVER (PARTITION BY Year_Month ORDER BY COUNT(*) DESC) AS RowNum
    FROM Clean_LastfmData
    GROUP BY Year_Month, Artist
)
SELECT 
    R.Year_Month,
    R.Artist,
    R.Scrobblings,
    T.TotalScrobblings,
	ROUND(CAST(R.Scrobblings AS FLOAT)/(T.TotalScrobblings) * 100 ,2) AS Percentage_total
FROM RankedArtists AS R
JOIN TotalCounts AS T ON R.Year_Month = T.Year_Month
WHERE R.RowNum = 1
ORDER BY Percentage_total DESC;
"""

# Ejecutar y cargar resultados
df = pd.read_sql(query, engine)
df.head()

,Year_Month,Artist,Scrobblings,TotalScrobblings,Percentage_total
0,2022-04,Last Days of Humanity,136,570,23.86
1,2016-07,1389,53,308,17.21
2,2021-06,Judas Iscariot,84,525,16.00
3,2016-02,Jarnvidr,93,594,15.66
4,2021-05,Judas Iscariot,87,588,14.80


### Plots